In [1]:
import pandas as pd

df = pd.read_excel('../../data/BioPubSum/BioPubSum_test_fill_filter.xlsx')
# df.head()

In [2]:
highlights = df['New_Highlight'].tolist()
abstracts = df['Abstract'].tolist()

#### prepare the chatgpt settings

In [3]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from tqdm import tqdm
import os

In [4]:
os.environ['OPENAI_API_KEY'] = 'sk-JO2fqzFAodIez6sGE76f61B7FeDc41E4A83e16574a3e28Ef'

In [5]:
# llm = OpenAI(model_name='gpt-3.5-turbo-16k',temperature=0)
llm = OpenAI(model_name='gpt-3.5-turbo-0613', temperature=0, openai_api_base='https://api.aimd5.com/v1')

e:\software\anaconda3\envs\pytorch\lib\site-packages\langchain\llms\openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
e:\software\anaconda3\envs\pytorch\lib\site-packages\langchain\llms\openai.py:751: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


#### zero-shot

In [6]:
template = 'Extract the main work:{input}\n'

prompt = PromptTemplate(
    input_variables = ['input'],
    template = template
)

chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
# toolong_texts = []

def generate(texts, args):
    for i in tqdm(range(len(texts))):
        text = texts[i]
        result = chain.run(text)
        # try:
        #     result = chain.run(text)
        # except Exception as e:
        #     toolong_texts.append(text)
        #     continue
        # args.append(result)
        args.append(result)

In [ ]:
from threading import Thread

g_res_1, g_res_2, g_res_3, g_res_4, g_res_5 = [],[],[],[],[]
pos = int(len(abstracts)/5)

t1 = Thread(target=generate, args=(abstracts[0:pos], g_res_1))
t2 = Thread(target=generate, args=(abstracts[pos:pos*2], g_res_2))
t3 = Thread(target=generate, args=(abstracts[pos*2:pos*3], g_res_3))
t4 = Thread(target=generate, args=(abstracts[pos*3:pos*4], g_res_4))
t5 = Thread(target=generate, args=(abstracts[pos*4:], g_res_5))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()

In [9]:
generated_highlights = g_res_1 + g_res_2 + g_res_3 + g_res_4 + g_res_5

In [ ]:
generated_highlights = []

for i in tqdm(range(len(abstracts))):
    text = abstracts[i]
    result = chain.run(text)
    generated_highlights.append(result)

In [10]:
df['generate'] = generated_highlights
df.to_excel('raw_prompt/abstract_(5)_bio_filter.xlsx', index=False)

#### few-shot

In [6]:
import pandas as pd
df_train = pd.read_excel('../../data/AIPubSumm/AIPubSumm_train.xlsx')

train_highlights = df_train['New_Highlight'].tolist()
train_abstracts = df_train['Abstract'].tolist()

In [7]:
from langchain import FewShotPromptTemplate

example_formatter_template = """
input:{input}\n
output:{output}
"""

example_prompt = PromptTemplate(
    input_variables= ['input','output'],
    template = example_formatter_template
)

In [8]:
instructions = [
    'Refine 4~5 innovative points of the scientific paper\
    abstract provided below in bullet point format without additional explanations.'
]

In [9]:
# prefix = 'Refine 4~5 innovative points of the scientific paper\
# abstract provided below in bullet point format without additional explanations.'
prefix = instructions[0]

##### random sample (pseudo input with demonstration)

In [ ]:
examples = []

for i in range(16):
    example = {}
    # example['input'] = train_abstracts[i]
    example['input'] = 'it just a test sentence'
    example['output'] = train_highlights[i]

    examples.append(example)

# examples

In [18]:
few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    # prefix = prefix,
    suffix = '\ninput:{input}\noutput:',
    input_variables = ['input'],
    example_separator = '\n',
)

In [19]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)
print(chain.run(abstracts[0]))

The recognition accuracy and execution time of a facial expression recognition system are improved in this study. Various techniques are utilized to achieve this improvement. The face detection component is implemented using the Viola-Jones descriptor. The detected face is down-sampled using the Bessel transform to reduce the feature extraction space and improve processing time. Gabor feature extraction techniques are employed to extract thousands of facial features representing various facial deformation patterns. An AdaBoost-based hypothesis is formulated to select a subset of the extracted features to speed up classification. The selected features are then fed into a well-designed 3-layer neural network classifier trained using a back-propagation algorithm. The system is trained and tested using datasets from the JAFFE and Yale facial expression databases. An average recognition rate of 96.83% and 92.22% is achieved in the JAFFE and Yale databases, respectively. The execution time f

In [20]:
generated_highlights = []

for i in tqdm(range(len(abstracts))):
    text = abstracts[i]
    result = chain.run(text)
    generated_highlights.append(result)

100%|██████████| 65/65 [16:32<00:00, 15.26s/it]  


In [17]:
df['generate'] = generated_highlights
df.to_excel('opendata_result/demonstration/number/abstract_(4)_ai_16shot_pseudo_noInstruct.xlsx', index=False)

##### random sample

In [ ]:
examples = []
# example = {}

# example['input'] = train_abstracts[0]
# example['output'] = train_highlights[0]

# examples.append(example)

for i in range(8):
    example = {}
    example['input'] = train_abstracts[i]
    example['output'] = train_highlights[i]

    examples.append(example)

examples

In [22]:
# random average

import numpy as np

# seed: 2020 2021 2022 2023
np.random.seed(2023)

random_array = np.random.choice(len(train_abstracts), size=6, replace=False)

examples = []

for i in random_array:
    example = {}
    example['input'] = train_abstracts[i]
    example['output'] = train_highlights[i]

    examples.append(example)

In [23]:
few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = '\ninput:{input}\noutput:',
    input_variables = ['input'],
    example_separator = '\n',
)

chain = LLMChain(llm=llm, prompt=few_shot_prompt)

In [55]:
toolong_texts = []

def generate(texts, args):
    # generate_results = list(args)

    for i in tqdm(range(len(texts))):
        text = texts[i]
        try:
            result = chain.run(text)
        except Exception as e:
            toolong_texts.append(text)
            continue
        # generate_results.append(result)
        args.append(result)

In [ ]:
from threading import Thread

g_res_1, g_res_2, g_res_3, g_res_4, g_res_5 = [],[],[],[],[]

pos = int(len(abstracts)/5)

t1 = Thread(target=generate, args=(abstracts[0:pos], g_res_1))
t2 = Thread(target=generate, args=(abstracts[pos:pos*2], g_res_2))
t3 = Thread(target=generate, args=(abstracts[pos*2:pos*3], g_res_3))
t4 = Thread(target=generate, args=(abstracts[pos*3:pos*4], g_res_4))
t5 = Thread(target=generate, args=(abstracts[pos*4:], g_res_5))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()

generated_highlights = g_res_1 + g_res_2 + g_res_3 + g_res_4 + g_res_5

In [ ]:
# another way: single thread

generated_highlights = []

for i in tqdm(range(len(abstracts))):
    text = abstracts[i]
    result = chain.run(text)
    generated_highlights.append(result)

In [25]:
df['generate'] = generated_highlights
df.to_excel('opendata_result/demonstration/random/abstract_(4)_ai_6shot_random4.xlsx', index=False)

##### clostest example

based on rouge metric

In [10]:
import json

with open('opendata_result/demonstration/ai_rouge.json', 'r') as f:
    rouge_values = json.load(f)

In [11]:
new_rouge_values = []

for items in rouge_values:
    items = [item['rouge-2']['f'] for item in items]
    item_dict = {}
    for i, item in enumerate(items):
        item_dict[i] = item
    item_dict = sorted(item_dict.items(), key=lambda x:x[1], reverse=True)
    new_rouge_values.append(item_dict)

new_rouge_values[1][:5]

[(180, 0.06924643584521385),
 (192, 0.062091503267973865),
 (131, 0.0588235294117647),
 (60, 0.05842696629213483),
 (52, 0.056497175141242945)]

In [12]:
import rouge

def find_closest_rouge_demo(i,  k=5, reverse=False):
    rouge_value = new_rouge_values[i][:k]
    max_idxs = [item[0] for item in rouge_value]
    
    if reverse:
        max_idxs.reverse()
        
    # construct example
    examples = []
    for idx in max_idxs:
        example = {}

        example['input'] = train_abstracts[idx]
        example['output'] = train_highlights[idx]

        examples.append(example)

    # construct few_shot_prompt
    few_shot_prompt = FewShotPromptTemplate(
        examples = examples,
        example_prompt = example_prompt,
        prefix = prefix,
        suffix = '\ninput: {input}\nanswer:',
        input_variables = ['input'],
        example_separator = '\n',
    )

    return few_shot_prompt    

based on embedding

In [13]:
import json
import numpy as np

with open('opendata_result/demonstration/ai_train.json', 'r') as f:
    train_embeddings = json.load(f)

with open('opendata_result/demonstration/ai_test.json', 'r') as f:
    test_embeddings = json.load(f)

In [14]:
def cal_sim(a, b):
    dot_product = np.dot(a, b)

    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    cos_similarity = dot_product / (norm_a * norm_b)

    return cos_similarity

In [15]:
sim_values = []

for i, item in enumerate(test_embeddings):
    item = np.array(item)
    sim_value = {}
    for j, test_embed in enumerate(train_embeddings):
        test_embed = np.array(test_embed)
        sim_value[j] = cal_sim(item, test_embed)
    
    sim_value = sorted(sim_value.items(), key=lambda x:x[1], reverse=True)
    
    sim_values.append(sim_value)

In [16]:
def find_closest_embedding_demo(i, k=5, reverse=False):
    test_sample = abstracts[i]
    sim_value = sim_values[i]

    max_idxs = [sim_value[j][0] for j in range(k)]

    if reverse:
        max_idxs.reverse()
        
    # construct example
    examples = []

    for idx in max_idxs:
        example = {}

        example['input'] = train_abstracts[idx]
        example['output'] = train_highlights[idx]

        examples.append(example)

    # construct few_shot_prompt
    few_shot_prompt = FewShotPromptTemplate(
        examples = examples,
        example_prompt = example_prompt,
        prefix = prefix,
        suffix = '\ninput: {input}\nanswer:',
        input_variables = ['input'],
        example_separator = '\n',
    )

    return few_shot_prompt    

In [22]:
generated_highlights = []

for i in tqdm(range(len(abstracts))):
    text = abstracts[i]
    few_shot_prompt = find_closest_rouge_demo(i, k=6, reverse=True)
    # few_shot_prompt = find_closest_embedding_demo(i, k=1)
    
    chain = LLMChain(llm=llm, prompt=few_shot_prompt)
    
    result = chain.run(text)
    generated_highlights.append(result)

100%|██████████| 65/65 [03:48<00:00,  3.51s/it]


In [23]:
df['generate'] = generated_highlights
df.to_excel('opendata_result/demonstration/similarity/abstract_(4)_ai_6shot_clostRouge_reverse.xlsx', index=False)